In [1]:
import pandas as pd

nodes = pd.read_csv("neo4j_query_table_data_2025-11-19.csv")
edges = pd.read_csv("graf_table.csv")

print(nodes.head())
print(edges.head())

   node_id        label  community  community_size  tarif_seharusnya  \
0        0      Patient       1579              65               NaN   
1        1        Claim       1579              65         3142869.0   
2        2    Diagnosis       1579              65               NaN   
3        3    Procedure       1579              65               NaN   
4        4  ServiceType         33             321               NaN   

   tarif_diklaim  lama_rawat  status_klaim  is_fraud  
0            NaN         NaN           NaN       NaN  
1      3293000.0         3.0  Diverifikasi       1.0  
2            NaN         NaN           NaN       NaN  
3            NaN         NaN           NaN       NaN  
4            NaN         NaN           NaN       NaN  
   source  target       relation
0       0       1     MADE_CLAIM
1       0     256     MADE_CLAIM
2       0     423     MADE_CLAIM
3       0    1347     MADE_CLAIM
4       1       2  HAS_DIAGNOSIS


In [2]:
nodes["is_fraud"] = nodes["is_fraud"].fillna(0).astype(int)

In [4]:
from sklearn.model_selection import train_test_split

train_nodes, test_nodes = train_test_split(
    nodes,
    test_size=0.3,
    stratify=nodes["is_fraud"],
    random_state=42
)

In [5]:
train_node_ids = set(train_nodes["node_id"])
test_node_ids  = set(test_nodes["node_id"])

In [8]:
train_edges = edges[
    edges["source"].isin(train_node_ids) & edges["target"].isin(train_node_ids)
]
test_edges = edges[
    edges["source"].isin(test_node_ids) & edges["target"].isin(test_node_ids)
]
print(f"Train nodes: {len(train_nodes)}, Train edges: {len(train_edges)}")
print(f"Test nodes: {len(test_nodes)}, Test edges: {len(test_edges)}")

Train nodes: 1315, Train edges: 2452
Test nodes: 564, Test edges: 441


In [9]:
# Mapping untuk seluruh node
all_nodes = pd.concat([train_nodes, test_nodes])
mapping = {old: new for new, old in enumerate(all_nodes["node_id"])}

# Tambahkan kolom "mapped_id"
all_nodes["mapped_id"] = all_nodes["node_id"].map(mapping)
train_nodes["mapped_id"] = train_nodes["node_id"].map(mapping)
test_nodes["mapped_id"] = test_nodes["node_id"].map(mapping)

# Map edges
edges["source_mapped"] = edges["source"].map(mapping)
edges["target_mapped"] = edges["target"].map(mapping)

In [10]:
train_nodes.to_csv("train_nodes.csv", index=False)
test_nodes.to_csv("test_nodes.csv", index=False)

train_edges.to_csv("train_edges.csv", index=False)
test_edges.to_csv("test_edges.csv", index=False)

print("✅ Berhasil membuat train/test nodes & edges")

✅ Berhasil membuat train/test nodes & edges


: 